<a href="https://colab.research.google.com/github/Ghonem22/Learning/blob/main/Python3%20object%20oriented%20programming/Ch5%2C%20When%20to%20Use%20Object-oriented%20Programming/When_to_Use_Object_oriented_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH5: When to Use Object-oriented Programming

## Treat objects as objects

### objects are things that have both data and behavior:

* If we are working only with data, we can sort it in a list, set, dictionary, or some other Python data structure.

* if we are working only with behavior, but no stored data, a simple function is more suitable.

---
We can often start our Python programs by storing data in a few variables, we will later find that we are passing the same set of related variables to a set of functions. This is the time to think about grouping both variables and functions into a class.

### designing a program to model polygons in two dimensional space:

* start with each polygon being represented as a list of points.
* want to calculate the distance around the perimeter of the polygon: To do this, we also need a function

we clearly recognize that a polygon class could encapsulate the list of points (data) and the perimeter function (behavior).

In [ ]:
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def distance(self, p2):
        return math.sqrt((self.x-p2.x)**2 + (self.y-p2.y)**2)
    
class Polygon:
    
    def __init__(self, points=None):
        points = points if points else []
        self.vertices = []
        
        # take list of tubles, convert each to Point object, and then add to vertices
        for point in points:
            if isinstance(point, tuple):
                point = Point(*point)
                self.vertices.append(point)   
                
    def add_point(self, *kwargs):
        self.vertices.append((kwargs))
        
    def perimeter(self):
        perimeter = 0
        points = self.vertices + [self.vertices[0]]
        for i in range(len(self.vertices)):
            perimeter += points[i].distance(points[i+1])
        return perimeter

If we have new functions that accept a polygon argument, such as area(polygon) or point_in_polygon(polygon, x, y), or if we add other attributes to the polygon, such as color or texture  the benefits of the object-oriented code become increasingly obvious.

---
The distinction is a design decision, but in general, the more complicated a set of
data is, the more likely it is to have multiple functions specific to that data, and the
more useful it is to use a class with attributes and methods instead.

---
If we're only trying to calculate the perimeter of one polygon in the context of a much reater problem, using a function will probably be quickest to code and easier to use "one time only".

---
if our program needs to manipulate numerous polygons in a wide variety of ways (calculate perimeter, area, intersection with other polygons, move or scale them, and so on), we have most certainly identified an object; one that needs to be extremely versatile.

In [ ]:
poly = Polygon([(5,2),(6,4),5])


In [ ]:
poly.vertices

[<__main__.Point at 0x1eb923974c0>, <__main__.Point at 0x1eb923975e0>]

In [ ]:
poly.perimeter()

4.47213595499958

## Adding behavior to class data with properties

let's discuss some bad object-oriented theory:

For Example, Java teach us to never access attributes directly: The variables are prefixed with an underscore to suggest that they are private Then the get and set methods provide access to each variable.

In [ ]:
class Color:
    
    def __init__(self, rgb_value, name):
        self._rgb_value = rgb_value
        self._name = name
        
    def set_name(self, name):
        self._name = name
    def get_name(self):
        return self._name

In [ ]:
c = Color("#ff0000", "bright red")

In [ ]:
c.get_name()

'bright red'

In [ ]:
c.set_name("red")

In [ ]:
c.get_name()

'red'

But this is not nearly as readable as the direct access version that Python favors:

### So why would anyone insist upon the method-based syntax?

* someday we may want to add extra code when a value is set or retrieved.
* we might want to validate the value.

In [ ]:
# we could decide to change the set_name() method as follows:
def set_name(self, name):
    if not name:
        raise Exception("Invalid Name")
    self._name = name

### in Java and similar languages

if the code was written to do direct attribute access, and then later changed it to a method, we'd
have a problem
* anyone who had written code that accessed the attribute directly would now have to access the method. If they don't change the access style from attribute access to a function call, their code will be broken.

* This doesn't make much sense in Python since there isn't any real concept of private members!

### Python gives us the property keyword to make methods look like attributes.

can therefore write our code to use direct member access, and if we unexpectedly need to alter the implementation to do some calculation when getting or setting that attribute's value, we can do so without changing the interface.

In [ ]:
class Color:
    
    def __init__(self, rgb_value, name):
        self.rgb_value = rgb_value
        self._name = name
        
    def _set_name(self, name):
        if not name:
            raise Exception("Invalid Name")
        self._name = name
        
    def _get_name(self):
        return self._name
    name = property(_get_name, _set_name)

In [ ]:
c = Color("#0000ff", "bright red")

In [ ]:
c.name

'bright red'

In [ ]:
c.name = ""

Exception: Invalid Name

So, if we'd previously written code to access the name attribute, and then changed
it to use our property object, the previous code would still work, unless it was
sending an empty property value, which is the behavior we wanted to forbid in
the first place. Success!

People can still access the _name attribute directly and set it to an empty
string if they want to.

### Properties in detail

* In practice, properties are normally only defined with the first two parameters: the getter and setter functions.

* If we want to supply a docstring for a property, we can define it on the getter function; the property proxy will copy it into its own docstring.

* The deletion function is often left empty because object attributes are rarely deleted.

* If a coder does try to delete a property that doesn't have a deletion function specified, it will raise an exception.

* Therefore, if there is a legitimate reason to delete our property, we should supply that function.

In [ ]:
class Silly:
    def _get_silly(self):
        print("You are getting silly")
        return self._silly
    def _set_silly(self, value):
        print("You are making silly {}".format(value))
        self._silly = value
    def _del_silly(self):
        print("Whoah, you killed silly!")
        del self._silly
        
    # The property here is related by the object "silly", the coreesponding method is fired when we
    # set/ get/ del the object
    silly = property(_get_silly, _set_silly, _del_silly, "This is a silly property")

In [ ]:
s = Silly()

In [ ]:
s.silly = "funny"

You are making silly funny


In [ ]:
s.silly

You are getting silly


'funny'

In [ ]:
del s.silly

Whoah, you killed silly!


### Decorators – another way to create properties

The property function can be used with the decorator syntax to turn a get function into a property:


In [ ]:
class Foo:
    @property
    def foo(self):
        return "bar"
    
# This is equivalent to the previous foo = property(foo)

    @foo.setter
    def foo(self, value):
        self._foo = value
        
    @foo.deleter
    def foo(self):
        del self._silly

In [ ]:
# This class operates exactly the same as our earlier version
class Silly:
    @property
    def silly(self):
        "This is a silly property"
        print("You are getting silly")
        return self._silly
    
    @silly.setter
    def silly(self, value):
        print("You are making silly {}".format(value))
        self._silly = value
        
    @silly.deleter
    def silly(self):
        print("Whoah, you killed silly!")
        del self._silly


In [ ]:
s = Silly()
s.silly = "funny"

You are making silly funny


In [ ]:
s.silly

You are getting silly


'funny'

In [ ]:
del s.silly

Whoah, you killed silly!


### Deciding when to use properties

The only difference between an attribute and a property is that we can invoke custom actions automatically when a property is retrieved, set, or deleted.

---
#### For Example: caching a value that is difficult to calculate or expensive to look up

1- The first time the value is retrieved, we perform the lookup or calculation.
2- Then we could locally cache the value as a private attribute on our object.
3- the next time the value is requested, we return the stored data.

---
Here's how we might cache a web page:

In [ ]:
from urllib.request import urlopen

class WebPage:
    def __init__(self, url):
        self.url = url
        self._content = None
        
    @property
    def content(self):
        if not self._content:
            print("Retrieving New Page...")
            self._content = urlopen(self.url).read()
        return self._content

In [ ]:
import time
webpage = WebPage("http://ccphillips.net/")

In [ ]:
now = time.time()
content1 = webpage.content
time.time() - now

Retrieving New Page...


2.3337314128875732

In [ ]:
now = time.time()
content2 = webpage.content
time.time() - now

0.0

In [ ]:
content2 == content1

True

**Custom getters are also useful for attributes that need to be calculated on the fly, based on other object attributes. For example, we might want to calculate the average for a list of integers:**

In [ ]:
class AverageList(list):
    @property
    def average(self):
        return sum(self) / len(self)

In [ ]:
x = AverageList([10,20,30,40,50])

In [ ]:
x.average

30.0

**Custom setters are useful for validation, as we've already seen, but they can also
be used to proxy a value to another location. For example, we could add a content
setter to the WebPage class that automatically logs into our web server and uploads
a new page whenever the value is set.**

## Manager objects

Let's take a look at designing higher-level objects: the kinds of objects that manage other objects.

---
Management objects are more like office managers; they don't do the actual "visible" work out on the floor, but without them, there would be no communication.

---
the attributes on a management class tend to refer to other objects that do the "visible" work; the behaviors on such a class delegate to those other classes at the right time, and pass messages between them.

---
### Example, write a program that does a find and replace action for text files stored in a compressed ZIP file.

We need objects to represent the ZIP file and each individual text file (in Python standard library).

#### The manager object will be responsible for ensuring three steps occur in order:

1. Unzipping the compressed file.
2. Performing the find and replace action.
3. Zipping up the new files.

In [ ]:
import sys
import shutil
import zipfile
from pathlib import Path

class ZipReplace:
    def __init__(self, filename, search_string, replace_string):
        self.filename = filename
        self.search_string = search_string
        self.replace_string = replace_string
        self.temp_directory = Path("unzipped-{}".format(filename))

    def zip_find_replace(self):
        self.unzip_files()
        self.find_replace()
        self.zip_files()
    
    def unzip_files(self):
        self.temp_directory.mkdir()
        with zipfile.ZipFile(self.filename) as zip:
            zip.extractall(str(self.temp_directory))
            
    def find_replace(self):
        for filename in self.temp_directory.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.search_string, self.replace_string)
            with filename.open("w") as file:
                file.write(contents)
                
    def zip_files(self):
        with zipfile.ZipFile(self.filename, 'w') as file:
            for filename in self.temp_directory.iterdir():
                file.write(str(filename), filename.name)
        shutil.rmtree(str(self.temp_directory))

# # to run the program from the command line by passing the zip filename, search string, and replace string as arguments:
# if __name__ == "__main__":
#     ZipReplace(*sys.argv[1:4]).zip_find_replace()

# python zipsearch.py hello.zip hello hi    

### Why don't we use just a script ?

1. Readability: The code for each step is in a self-contained unit that is easy to read and understand.

2. Extensibility: If a subclass wanted to use compressed TAR files instead of ZIP files, it could override the zip and unzip methods without having to duplicate the find_replace method.

3. Partitioning: An external class could create an instance of this class and call the find_replace method directly on some folder without having to zip the content.

## Removing duplicate code

why is duplicate code a bad thing?

    All reasons boil down to readability and maintainability


### In practice

#### In the previous example, we want to scale all the images in a ZIP file to 640 x 480:

The first impulse might be to save a copy of that file and change the find_replace method to scale_image or something similar.

---

#### We will use inheritance, in case we want in the future to apply another functionality
We will convert our clss to superclass

In [ ]:
import sys
import shutil
import zipfile
from pathlib import Path

class ZipProcessor:
    def __init__(self, zipname):
        
        self.zipname = zipname
        self.temp_directory = Path("unzipped-{}".format(zipname[:-4]))

    def process_zip(self):
        self.unzip_files()
        self.process_files()
        self.zip_files()
    
    def unzip_files(self):
        self.temp_directory.mkdir()
        with zipfile.ZipFile(self.zipname) as zip:
            zip.extractall(str(self.temp_directory))
            
               
    def zip_files(self):
        with zipfile.ZipFile(self.zipname, 'w') as file:
            for filename in self.temp_directory.iterdir():
                file.write(str(filename), filename.name)
        shutil.rmtree(str(self.temp_directory))


In [ ]:
import sys
import os


class ZipReplace(ZipProcessor):
    def __init__(self, filename, search_string, replace_string):
        super().__init__(filename)
        self.search_string = search_string
        self.replace_string = replace_string
        
    def process_files(self):
        # perform a search and replace on all files in the temporary directory
        for filename in self.temp_directory.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.search_string, self.replace_string)
            with filename.open("w") as file:
                file.write(contents)
                
# if __name__ == "__main__":
#     ZipReplace(*sys.argv[1:4]).process_zip()

# python ZipReplace.py hello.zip hello hi    

**This program isn't functionally different from the one we started with! But it's much easier for us to write other classes That do diffrent process, such as: unzipping file, scale photos, then zip files again.**

In [ ]:
import sys
from PIL import Image


class ScaleZip(ZipProcessor):
    def process_files(self):
        '''Scale each image in the directory to 640x480'''
        for filename in self.temp_directory.iterdir():
            im = Image.open(str(filename))
            scaled = im.resize((640, 480))
            scaled.save(str(filename))
            
# if __name__ == "__main__":
#     ScaleZip(*sys.argv[1:4]).process_zip()            

## Case Study

when should I choose an object versus a built-in type?

---
We'll be modeling a Document class that might be used in a text editor or word processor.

---
What objects, functions, or properties should it have?

---
strings aren't mutable in python (able to be changed), So we will use character based code for simplicity. 

---
Document class would need to know the current cursor position within the list, and should probably also store a filename for the document.

---
### what methods should it have?

 * inserting, deleting, and selecting characters, cut, copy, paste, the selection, and saving or closing the document.
 
 * str filenames, int cursor positions, and a list of characters

In [ ]:
class Document:
    def __init__(self):
        self.filename = ''
        self.characters = []
        self.cursor = 0
        
    def insert(self, character):
        self.characters.insert(self.cursor, character )
        self.forward()
        
    def delete(self):
        del self.characters[ self.cursor -1]
        self.back()
    
    def save(self):
        with open(self.filename, 'w') as f:
            f.write("".join(self.characters))
            
    def forward(self):
        self.cursor += 1
        
    def back(self):
        self.cursor -= 1

**Let's try our class:**

In [ ]:
document = Document()
document.filename = 'file.txt'

In [ ]:
document.insert('h')
document.insert('l')
document.insert('l')
document.insert('o')
document.insert(' ')
document.insert('w')
document.insert('r')
document.insert('l')
document.insert('d')

In [ ]:
"".join(document.characters)

'hllo wrld'

In [ ]:
document.cursor

9

In [ ]:
document.delete()

In [ ]:
document.cursor

8

In [ ]:
"".join(document.characters)

'hllo wrl'

In [ ]:
document.save()

### What if we want to connect the Home and End keys as well?

We could add more methods to the Document class that search forward or backwards for newline characters in the string and jump to them.

but if we did that for every possible movement action (move by words, move by sentences, Page Up, Page Down, end of line, beginning of whitespace, and more), the class would be huge

In [ ]:
class Cursor:
    def __init__(self, document):
        self.document = document
        self.position = 0
        
    def forward(self):
        self.position += 1
        
    def back(self):
        self.position -= 1
        
    def home(self):
        while self.document.characters[self.position-1] != '\n':
            self.position -= 1
            if self.position == 0:
                # Got to beginning of file before newline
                break
        
    def end(self):
        while self.position < len(self.document.characters) and self.document.characters[self.position] != '\n':
            self.position += 1

In [ ]:
class Document:
    def __init__(self):
        self.characters = []
        self.cursor = Cursor(self)                # Composition
        self.filename = ''
        
    def insert(self, character):
        self.characters.insert(self.cursor.position, character)
        self.cursor.forward()
        
    def delete(self):
        del self.characters[self.cursor.position]
        
    def save(self):
        f = open(self.filename, 'w')
        f.write(''.join(self.characters))
        f.close()
        
    @property
    def string(self):
        return "".join(self.characters)

In [ ]:
document = Document()
document.insert('h')
document.insert('l')
document.insert('l')
document.insert('o')
document.insert('\n')
document.insert('w')
document.insert('r')
document.insert('l')
document.insert('d')

In [ ]:
document.cursor.home()
document.insert('*')


In [ ]:
document.characters

['h', 'l', 'l', 'o', '\n', '*', 'w', 'r', 'l', 'd']

In [ ]:
print("".join(document.characters))

hllo
*wrld


### since we've been using that string join function a lot, we can add it as a property to the Document

In [ ]:
document.string

'hllo\n*wrld'

### let's extend it to work for rich text; text that can have bold, underlined, or italic characters.

* if the Character class won't have any methods, we should use one of the many Python data structures instead.

* we might want to do things with characters, such as delete or copy them, but those are things that need to be handled at the Document level.

---
There is a very important special method on the object class called __str__ which is used in string manipulation functions like print and the str constructor to convert any class to a string.


The default implementation does some boring stuff like printing the name of the module and class and its address in memory.


But if we override it, we can make it print whatever we like.


we could make it prefix characters with special characters
to represent whether they are bold, italic, or underlined. So, we will create a class
to represent a character, and here it is:

In [ ]:
class Character:
    def __init__(self, character,bold=False, italic=False, underline=False):
        assert len(character) == 1
        self.character = character
        self.bold = bold
        self.italic = italic
        self.underline = underline
        
    def __str__(self):
        bold = "*" if self.bold else ''
        italic = "/" if self.italic else ''
        underline = "_" if self.underline else ''
        return bold + italic + underline + self.character

In the Document class, we add these two lines at the beginning of the insert method

---
In addition, we need to modify the string property on Document to accept the new Character values. All we need to do is call str() on each character before we join it:

In [ ]:
class Document:
    def __init__(self):
        self.characters = []
        self.cursor = Cursor(self)
        self.filename = ''
        
    def insert(self, character):
        
        '''
        check whether the character being passed in is a Character or a str. 
        If it is a string, it is wrapped in a Character class so all objects
        in the list are Character objects.
        '''
        if not hasattr(character, 'character'):
            character = Character(character)       # Composition
            
        self.characters.insert(self.cursor.position, character)
        self.cursor.forward()
        
    def delete(self):
        del self.characters[self.cursor.position]
        
    def save(self):
        f = open(self.filename, 'w')
        f.write(''.join(self.characters))
        f.close()
        
    @property
    def string(self):
        return "".join((str(c) for c in self.characters))

Finally, we also need to check Character.character at home and end methods

In [ ]:
class Cursor:
    def __init__(self, document):
        self.document = document
        self.position = 0
        
    def forward(self):
        self.position += 1
        
    def back(self):
        self.position -= 1
        
    def home(self):
        while self.document.characters[self.position-1].character != '\n':
            self.position -= 1
            if self.position == 0:
                # Got to beginning of file before newline
                break
        
    def end(self):
        while self.position < len(self.document.characters) and self.document.characters[self.position].character != '\n':
            self.position += 1

In [ ]:
d = Document()

In [ ]:
d.insert('h')
d.insert('e')
d.insert(Character('l', bold=True))
d.insert(Character('l', bold=True))
d.insert('o')
d.insert('\n')
d.insert(Character('w', italic=True))
d.insert(Character('o', italic=True))
d.insert(Character('r', underline=True))
d.insert('l')
d.insert('d')

In [ ]:
print(d.string)

he*l*lo
/w/o_rld


In [ ]:
d.characters[0].underline = True

In [ ]:
print(d.string)

_he*l*lo
/w/o_rld
